# 🔎 Loki (multi-tenant) → Parquet → Episodes

This notebook pulls logs from **application**, **infrastructure**, and **audit** tenants on an OpenShift Loki gateway,
normalizes them with a JSON-first projector, writes `data/unified_logs/latest.parquet` & CSV, and builds 10‑minute episodes.
It includes robust auth/SSL handling, nanosecond timestamp parsing, and tenant-specific helpers.


## 0) Setup

In [1]:
# If needed, install deps (uncomment once)
# %pip install --quiet pandas numpy requests pyarrow


In [2]:
from pathlib import Path
import os, pandas as pd, numpy as np

# --- Storage locations
DATA_DIR = Path("data"); DATA_DIR.mkdir(parents=True, exist_ok=True)
UNIFIED_DIR = DATA_DIR / "unified_logs"; UNIFIED_DIR.mkdir(exist_ok=True, parents=True)
INCIDENTS_DIR = Path("incidents"); INCIDENTS_DIR.mkdir(exist_ok=True, parents=True)
RULES_DIR = Path("rules"); RULES_DIR.mkdir(exist_ok=True, parents=True)

# --- Time window (adjust as needed)
END   = pd.Timestamp.utcnow()
START = END - pd.Timedelta("90min")
print("Window:", START, "→", END)


Window: 2025-09-10 17:40:09.196834+00:00 → 2025-09-10 19:10:09.196834+00:00


## 1) Loki helpers (tenant‑aware + token + SSL toggle)

In [ ]:
# Cell 1.1 — config + session + diagnostics
import pandas as pd, requests, urllib3

# ---- Config (set env vars or edit here) ----
LOKI_BASE       = os.environ.get("LOKI_BASE", "https://logging-loki-openshift-logging.apps.rhoai.ocp-poc-demo.com")
LOKI_TOKEN      = os.environ.get("LOKI_TOKEN", "<REDACTED>")                   # e.g. export LOKI_TOKEN="$(oc whoami -t)"
LOKI_INSECURE   = os.environ.get("LOKI_INSECURE", "true").lower() in ("1","true","yes")
LOKI_ORG_ID     = os.environ.get("LOKI_ORG_ID")                  # some gateways require X-Scope-OrgID
LOKI_BASIC_USER = os.environ.get("LOKI_BASIC_USER")
LOKI_BASIC_PASS = os.environ.get("LOKI_BASIC_PASS")

if LOKI_INSECURE:
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

_session = requests.Session()
_default_headers = {"Accept": "application/json"}
if LOKI_TOKEN:
    _default_headers["Authorization"] = f"Bearer {LOKI_TOKEN}"
if LOKI_ORG_ID:
    _default_headers["X-Scope-OrgID"] = LOKI_ORG_ID

def _debug_response(resp):
    ct = resp.headers.get("Content-Type", "")
    preview = (resp.text or "")[:500]
    return f"HTTP {resp.status_code} CT={ct} URL={resp.url}\nBody (first 500):\n{preview}"


In [4]:
# Cell 1.2 — tenant ping / labels / query_range (nanosecond safe)
def loki_ping_tenant(tenant: str):
    url = f"{LOKI_BASE.rstrip('/')}/api/logs/v1/{tenant}/loki/api/v1/labels"
    r = _session.get(url, headers=_default_headers, timeout=30, verify=not LOKI_INSECURE, allow_redirects=False)
    if not r.ok or "application/json" not in r.headers.get("Content-Type","").lower():
        raise RuntimeError(f"Ping failed for tenant={tenant}:\n" + _debug_response(r))
    return r.json()

def loki_labels_tenant(tenant: str):
    url = f"{LOKI_BASE.rstrip('/')}/api/logs/v1/{tenant}/loki/api/v1/labels"
    r = _session.get(url, headers=_default_headers, timeout=30, verify=not LOKI_INSECURE, allow_redirects=False)
    r.raise_for_status()
    return r.json().get("data", [])

def loki_query_range_tenant(tenant: str, expr, start_ts, end_ts, step='15s', limit=5000, direction='forward'):
    """
    Robust query_range that ALWAYS returns a pandas DataFrame (possibly empty).
    It handles redirects, non-JSON, odd timestamp formats, and unexpected payloads.
    """
    import pandas as pd

    url = f"{LOKI_BASE.rstrip('/')}/api/logs/v1/{tenant}/loki/api/v1/query_range"
    params = {
        "query": expr,
        "start": int(pd.Timestamp(start_ts).value),  # ns
        "end": int(pd.Timestamp(end_ts).value),
        "step": step,
        "limit": str(limit),
        "direction": direction,
    }
    auth = (LOKI_BASIC_USER, LOKI_BASIC_PASS) if (LOKI_BASIC_USER and LOKI_BASIC_PASS) else None
    r = _session.get(url, params=params, headers=_default_headers, timeout=60,
                     verify=not LOKI_INSECURE, auth=auth, allow_redirects=False)

    # Redirects usually mean OAuth login page
    if r.is_redirect or r.status_code in (301,302,303,307,308):
        raise RuntimeError(f"Auth redirect for tenant={tenant}.\n" + _debug_response(r))

    # Must be OK and JSON
    if not r.ok:
        raise RuntimeError(f"Loki query_range failed for tenant={tenant}:\n" + _debug_response(r))
    if "application/json" not in (r.headers.get("Content-Type", "")).lower():
        raise RuntimeError(f"Non-JSON response for tenant={tenant}:\n" + _debug_response(r))

    # Parse JSON safely
    try:
        payload = r.json()
    except Exception:
        # Return empty DF but surface the raw body in an error to the caller
        raise RuntimeError(f"Could not parse JSON for tenant={tenant}:\n" + _debug_response(r))

    # Defensive: tolerate odd shapes
    data = {}
    if isinstance(payload, dict):
        data = payload.get("data") or {}
    results = data.get("result") if isinstance(data, dict) else None
    if results is None:
        # Return empty DF rather than None
        return pd.DataFrame(columns=["ts", "line"])

    def _parse_ns(ts_val):
        s = str(ts_val)
        try:
            ns = int(s)
        except ValueError:
            ns = int(float(s))  # handles "1.757e+18"
        return pd.to_datetime(ns, unit="ns", utc=True)

    rows = []
    for series in results or []:
        labels = series.get("metric", {})
        # Loki can return either "values" (range vector) or "value" (instant)
        values = series.get("values") or []
        if not values and "value" in series:
            values = [series["value"]]
        for ts, line in values:
            rows.append({"ts": _parse_ns(ts), "line": line, **labels})

    return pd.DataFrame(rows, columns=["ts","line", *({} if not rows else rows[0].keys())])

def tenant_wildcard_selector(tenant: str) -> str:
    """
    Returns a selector like {k8s_namespace_name=~".+"} using a label that exists in the tenant.
    Loki requires at least one matcher that cannot match empty.
    """
    try:
        labels = set(loki_labels_tenant(tenant) or [])
    except Exception:
        labels = set()

    # Try likely k8s labels first, then fall back.
    prefs = [
        "k8s_namespace_name", "kubernetes_namespace_name",
        "k8s_pod_name", "kubernetes_pod_name",
        "k8s_node_name", "kubernetes_host",
        "log_type", "namespace", "pod", "node",
        "job", "filename"
    ]
    for key in prefs:
        if key in labels:
            return f'{{{key}=~".+"}}'  # non-empty regex satisfies Loki
    # absolute fallback: use whatever first label exists
    if labels:
        key = sorted(labels)[0]
        return f'{{{key}=~".+"}}'
    # last resort (shouldn’t happen if tenant has data)
    return '{job=~".+"}'

    auth = (LOKI_BASIC_USER, LOKI_BASIC_PASS) if (LOKI_BASIC_USER and LOKI_BASIC_PASS) else None
    r = _session.get(url, params=params, headers=_default_headers, timeout=60,
                     verify=not LOKI_INSECURE, auth=auth, allow_redirects=False)
    if r.is_redirect or r.status_code in (301,302,303,307,308):
        raise RuntimeError(f"Auth redirect for tenant={tenant}. Token/headers likely missing.\n" + _debug_response(r))
    if not r.ok:
        raise RuntimeError(f"Loki query_range failed for tenant={tenant}:\n" + _debug_response(r))
    if "application/json" not in r.headers.get("Content-Type","").lower():
        raise RuntimeError(f"Non-JSON response for tenant={tenant}:\n" + _debug_response(r))

    payload = r.json()
    data = payload.get("data", {}).get("result", [])

    def _parse_ns(ts_val):
        s = str(ts_val)
        try: ns = int(s)
        except ValueError: ns = int(float(s))
        return pd.to_datetime(ns, unit="ns", utc=True)

    rows = []
    for series in data:
        labels = series.get("metric", {})
        for ts, line in series.get("values", []):
            rows.append({"ts": _parse_ns(ts), "line": line, **labels})
    return pd.DataFrame(rows)

# Quick tenant sanity
for t in ["application","infrastructure","audit"]:
    try:
        info = loki_ping_tenant(t)
        print(f"tenant={t} OK, ~{len(info.get('data', []))} labels")
    except Exception as e:
        print(f"tenant={t} ping failed:", e)


tenant=application OK, ~10 labels
tenant=infrastructure OK, ~11 labels
tenant=audit OK, ~4 labels


## 2) Pull logs from all tenants

In [5]:
# === Step 2 — SAFE multi-tenant fetch (always returns DataFrames) ===
import pandas as pd

def tenant_wildcard_selector(tenant: str) -> str:
    try:
        labels = set(loki_labels_tenant(tenant) or [])
    except Exception:
        labels = set()
    prefs = [
        "k8s_namespace_name","kubernetes_namespace_name","namespace",
        "k8s_pod_name","kubernetes_pod_name","pod",
        "k8s_node_name","kubernetes_host","node",
        "job","filename","log_type"
    ]
    for key in prefs:
        if key in labels:
            return f'{{{key}=~".+"}}'
    if labels:
        key = sorted(labels)[0]
        return f'{{{key}=~".+"}}'
    return '{job=~".+"}'

def safe_fetch(tenant: str, start, end, step="60s", limit=5000):
    try:
        _ = loki_ping_tenant(tenant)  # auth/scope sanity
        sel = tenant_wildcard_selector(tenant)
        print(f"[{tenant}] selector:", sel)
        df = loki_query_range_tenant(tenant, sel, start, end, step=step, limit=limit)
        # Guarantee a DataFrame with expected minimal columns
        if not isinstance(df, pd.DataFrame):
            print(f"[{tenant}] unexpected return type: {type(df)} → coercing to empty DataFrame")
            df = pd.DataFrame(columns=["ts","line"])
        if "ts" not in df.columns:
            df["ts"] = pd.NaT
        if "line" not in df.columns:
            df["line"] = ""
        print(f"[{tenant}] rows: {len(df)}")
        return df
    except Exception as e:
        print(f"[{tenant}] fetch failed → {e}")
        return pd.DataFrame(columns=["ts","line"])

# Wider window while debugging
END   = pd.Timestamp.utcnow()
START = END - pd.Timedelta("6h")

df_app   = safe_fetch("application",    START, END)
df_infra = safe_fetch("infrastructure", START, END)
df_audit = safe_fetch("audit",          START, END)

def _shape(df):
    return f"{type(df)} | shape={getattr(df,'shape',None)} | empty={getattr(df,'empty',None)}"
print("df_app   →", _shape(df_app))
print("df_infra →", _shape(df_infra))
print("df_audit →", _shape(df_audit))


[application] selector: {k8s_namespace_name=~".+"}
[application] rows: 5000
[infrastructure] selector: {k8s_namespace_name=~".+"}
[infrastructure] rows: 5000
[audit] selector: {k8s_node_name=~".+"}
[audit] rows: 5000
df_app   → <class 'pandas.core.frame.DataFrame'> | shape=(5000, 4) | empty=False
df_infra → <class 'pandas.core.frame.DataFrame'> | shape=(5000, 4) | empty=False
df_audit → <class 'pandas.core.frame.DataFrame'> | shape=(5000, 4) | empty=False


## 3) JSON‑first projector (namespace/pod/node from labels or JSON body)

In [6]:
# --- Replace your existing projector with this robust version ---
import json, re
import pandas as pd

def _maybe_json(s: str):
    if not isinstance(s, str): 
        return None
    s = s.strip()
    if not s or s[0] not in "{[": 
        return None
    try:
        return json.loads(s)
    except Exception:
        return None

def _get_any(obj, keys):
    for k in keys:
        cur = obj
        try:
            for part in k.split("."):
                if isinstance(cur, dict) and part in cur:
                    cur = cur[part]
                else:
                    raise KeyError
            return cur
        except Exception:
            continue
    return None

def _normalize_level(obj, line: str):
    if isinstance(obj, dict):
        v = _get_any(obj, ["level","severity","loglevel","lvl","logger_level"])
        if v is not None: return str(v).lower()
    s = (line or "").lower()
    if any(w in s for w in ["error","exception","fail","backoff","oomkilled","notready"]): return "error"
    if "warn" in s or "throttle" in s: return "warn"
    return "info"

def _extract_code(obj, line: str):
    if isinstance(obj, dict):
        v = _get_any(obj, ["status","status_code","code","http.status","response.status"])
        try:
            if v is not None: return int(v)
        except Exception:
            pass
    m = re.search(r"\s(1\d{2}|2\d{2}|3\d{2}|4\d{2}|5\d{2})\s", " " + (line or "") + " ")
    return int(m.group(1)) if m else None

def _extract_route(obj, line: str):
    if isinstance(obj, dict):
        v = _get_any(obj, ["path","route","url","request_path","http.path","request.url","endpoint"])
        if isinstance(v, str): return v.split("?")[0]
    m = re.search(r"\s(?:GET|POST|PUT|PATCH|DELETE)\s+(\S+)", " " + (line or "") + " ")
    return m.group(1) if m else None

def _to_text(v):
    if v is None: return ""
    if isinstance(v, str): return v
    try:
        return json.dumps(v, default=str)
    except Exception:
        return str(v)

def project_unified_stronger(df: pd.DataFrame, source_guess: str) -> pd.DataFrame:
    # 1) Coerce 'line' to text safely (fixes .str accessor errors)
    if "line" not in df.columns:
        line_text = pd.Series([""] * len(df), index=df.index)
    else:
        line_text = df["line"].map(_to_text).astype("string")  # ensure string dtype

    # 2) Prefer k8s label columns; else pull from JSON body
    ns_series   = df.get("k8s_namespace_name") or df.get("kubernetes_namespace_name") or df.get("namespace")
    pod_series  = df.get("k8s_pod_name")       or df.get("kubernetes_pod_name")       or df.get("pod")
    node_series = df.get("k8s_node_name")      or df.get("kubernetes_host")           or df.get("node")

    # Parse JSON after coercion
    objs = line_text.map(_maybe_json)

    def _label_or_json(series_or_none, json_keys):
        if series_or_none is not None:
            return series_or_none
        vals = []
        for o in objs:
            v = _get_any(o, json_keys) if isinstance(o, dict) else None
            vals.append(v)
        return pd.Series(vals, index=line_text.index)

    namespace = _label_or_json(ns_series,   ["kubernetes.namespace_name","k8s.namespace.name","k8s.ns","namespace"])
    pod       = _label_or_json(pod_series,  ["kubernetes.pod_name","k8s.pod.name","pod"])
    node      = _label_or_json(node_series, ["kubernetes.host","kubernetes.node_name","k8s.node.name","node"])

    # Features
    level = [_normalize_level(o, ln) for o, ln in zip(objs, line_text)]
    code  = [_extract_code(o, ln)    for o, ln in zip(objs, line_text)]
    route = [_extract_route(o, ln)   for o, ln in zip(objs, line_text)]

    container_restart = line_text.str.contains(r"\bRestarted container\b", case=False, na=False).astype(int)
    rollout_hit = line_text.str.contains(
        r"Scaled up replica set|deployment (created|updated|rolled out)|\brollout\b",
        case=False, na=False, regex=True
    ).astype(float)

    return pd.DataFrame({
        "ts": df.get("ts", pd.NaT),
        "source": source_guess,
        "namespace": namespace,
        "pod": pod,
        "node": node,
        "level": level,
        "verb": None,
        "code": code,
        "route": route,
        "msg": line_text.str.slice(0, 400),
        "container_restart": container_restart,
        "rollout_in_window": rollout_hit,
    })


## 4) Concat, write `latest.parquet`, and show quick stats

In [7]:
import pandas as pd
def _shape(df):
    if not isinstance(df, pd.DataFrame):
        return f"{type(df)}"
    return f"{df.shape}, empty={df.empty}"

print("df_app   →", _shape(df_app))
print("df_infra →", _shape(df_infra))
print("df_audit →", _shape(df_audit))

# If any have rows, show their columns so projector can map labels
for name, df in [("app", df_app), ("infra", df_infra), ("audit", df_audit)]:
    if isinstance(df, pd.DataFrame) and not df.empty:
        print(f"\n{name} columns:", list(df.columns))
        display(df.head(3))


df_app   → (5000, 4), empty=False
df_infra → (5000, 4), empty=False
df_audit → (5000, 4), empty=False

app columns: ['ts', 'line', 'ts', 'line']


,ts,line,ts,line
0,2025-09-10 13:10:23.900629823+00:00,"{""@timestamp"":""2025-09-10T13:10:23.900629823Z""...",2025-09-10 13:10:23.900629823+00:00,"{""@timestamp"":""2025-09-10T13:10:23.900629823Z""..."
1,2025-09-10 13:10:24.315266063+00:00,"{""@timestamp"":""2025-09-10T13:10:24.315266063Z""...",2025-09-10 13:10:24.315266063+00:00,"{""@timestamp"":""2025-09-10T13:10:24.315266063Z""..."
2,2025-09-10 13:10:24.315570277+00:00,"{""@timestamp"":""2025-09-10T13:10:24.315570277Z""...",2025-09-10 13:10:24.315570277+00:00,"{""@timestamp"":""2025-09-10T13:10:24.315570277Z""..."



infra columns: ['ts', 'line', 'ts', 'line']


,ts,line,ts,line
0,2025-09-10 13:10:26.097185665+00:00,"{""@timestamp"":""2025-09-10T13:10:26.097185665Z""...",2025-09-10 13:10:26.097185665+00:00,"{""@timestamp"":""2025-09-10T13:10:26.097185665Z""..."
1,2025-09-10 13:10:12.298212602+00:00,"{""@timestamp"":""2025-09-10T13:10:12.298212602Z""...",2025-09-10 13:10:12.298212602+00:00,"{""@timestamp"":""2025-09-10T13:10:12.298212602Z""..."
2,2025-09-10 13:10:14.845378415+00:00,"{""@timestamp"":""2025-09-10T13:10:14.845378415Z""...",2025-09-10 13:10:14.845378415+00:00,"{""@timestamp"":""2025-09-10T13:10:14.845378415Z""..."



audit columns: ['ts', 'line', 'ts', 'line']


,ts,line,ts,line
0,2025-09-10 13:10:10.411414683+00:00,"{""@timestamp"":""2025-09-10T13:10:10.411414683Z""...",2025-09-10 13:10:10.411414683+00:00,"{""@timestamp"":""2025-09-10T13:10:10.411414683Z""..."
1,2025-09-10 13:10:10.428389732+00:00,"{""@timestamp"":""2025-09-10T13:10:10.428389732Z""...",2025-09-10 13:10:10.428389732+00:00,"{""@timestamp"":""2025-09-10T13:10:10.428389732Z""..."
2,2025-09-10 13:10:10.428414040+00:00,"{""@timestamp"":""2025-09-10T13:10:10.428414040Z""...",2025-09-10 13:10:10.428414040+00:00,"{""@timestamp"":""2025-09-10T13:10:10.428414040Z""..."


In [8]:
from pathlib import Path
import pandas as pd

print("Sizes -> app/infra/audit:", len(df_app), len(df_infra), len(df_audit))
print("Empty flags ->", df_app.empty, df_infra.empty, df_audit.empty)
print("Using projector:", project_unified_stronger.__name__)

parts = []
if isinstance(df_app, pd.DataFrame) and not df_app.empty:
    parts.append(project_unified_stronger(df_app, "app"))
if isinstance(df_infra, pd.DataFrame) and not df_infra.empty:
    parts.append(project_unified_stronger(df_infra, "infra"))
if isinstance(df_audit, pd.DataFrame) and not df_audit.empty:
    parts.append(project_unified_stronger(df_audit, "audit"))

if parts:
    unified = pd.concat(parts, ignore_index=True)
else:
    print("No rows from any tenant in this window. Creating an empty unified frame.")
    unified = pd.DataFrame(columns=[
        "ts","source","namespace","pod","node","level","verb","code","route","msg",
        "container_restart","rollout_in_window"
    ])

# Dtypes & cleanup
if not unified.empty:
    unified["ts"] = pd.to_datetime(unified["ts"], utc=True, errors="coerce")
    unified = unified.dropna(subset=["ts"]).sort_values("ts").reset_index(drop=True)
    unified["container_restart"] = pd.to_numeric(unified["container_restart"], errors="coerce").fillna(0).astype("int64")
    unified["code"] = pd.to_numeric(unified["code"], errors="coerce")
else:
    # ensure ts exists with correct dtype
    unified["ts"] = pd.to_datetime(unified["ts"], utc=True, errors="coerce")

UNIFIED_DIR.mkdir(parents=True, exist_ok=True)
unified_path = UNIFIED_DIR / "latest.parquet"
csv_path     = UNIFIED_DIR / "latest.csv"

# Try Parquet; fall back to CSV if engine missing
try:
    import pyarrow  # noqa: F401
    unified.to_parquet(unified_path, index=False)
    wrote = f"parquet → {unified_path}"
except Exception as e:
    print("Parquet write failed:", e)
    print("Falling back to CSV.")
    unified.to_csv(csv_path, index=False)
    wrote = f"csv → {csv_path}\nHint: install Parquet engine with:  %pip install pyarrow"

print("Unified rows:", len(unified))
print("Wrote:", wrote)
print("Nulls by column:\n", unified.isna().mean().round(3))
if not unified.empty:
    print("Level distribution:\n", unified["level"].value_counts(dropna=False).head(10))
    print("HTTP status sample:\n", unified["code"].dropna().astype(int).value_counts().head(10))
display(unified.head(8))



Sizes -> app/infra/audit: 5000 5000 5000
Empty flags -> False False False
Using projector: project_unified_stronger


ValueError: Length of values (2) does not match length of index (5000)

## 5) Build 10‑minute episodes (namespace/pod/node groups)

In [ ]:
# === Step 5 — Build 10-minute episodes (safe when no data) ===
import pandas as pd

def build_episodes(df: pd.DataFrame, window="10min", keys=("namespace","pod","node")):
    if df is None or df.empty:
        return []
    df = df.copy()
    df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")
    df = df.dropna(subset=["ts"])
    if df.empty:
        return []

    df.set_index("ts", inplace=True)
    episodes = []
    for wstart, wdf in df.groupby(pd.Grouper(freq=window)):
        if wdf.empty:
            continue
        wend = wstart + pd.to_timedelta(window)
        grp_cols = [k for k in keys if k in wdf.columns]
        groups = dict(tuple(wdf.groupby(grp_cols, dropna=False))) if grp_cols else {"_": wdf}
        for gkey, gdf in groups.items():
            total = len(gdf)
            errors = (gdf.get("level","").eq("error")).sum()
            err_ratio = (errors/total) if total else 0.0
            restarts = gdf.get("container_restart", pd.Series([0]*total, index=gdf.index)).sum()
            http5xx = (gdf.get("code", pd.Series(dtype=float)) >= 500).sum()
            rollout = 1.0 if (gdf.get("rollout_in_window", pd.Series(dtype=float)) > 0).any() else 0.0
            entities = {}
            for col in ["namespace","pod","node"]:
                if col in gdf.columns:
                    vals = [v for v in gdf[col].astype(str).dropna().unique().tolist() if v and v != "None"]
                    if vals:
                        entities[col] = vals
            episodes.append({
                "episode_id": f"{int(wstart.value)}::{hash(str(gkey)) & 0xfffffff:07x}",
                "start": wstart,
                "end": wend,
                "entities": entities,
                "features": {
                    "count": float(total),
                    "error_ratio": float(err_ratio),
                    "restarts": float(restarts),
                    "http5xx": float(http5xx),
                    "rollout_in_window": rollout,
                },
            })
    return episodes

# Build episodes
eps = build_episodes(unified, window="10min")

# Summarize safely
if not eps:
    print("No episodes built. Likely because `unified` is empty or the window has no logs.")
    print("Tip: widen the window (e.g., START = END - pd.Timedelta('24h')) and re-fetch;")
    print("     or re-run the diagnostics selector step to ensure your queries return rows.")
    epi_dbg = pd.DataFrame(columns=["id","count","error_ratio","restarts","http5xx","rollout_in_window",
                                    "ent_namespace","ent_pod","ent_node","start","end"])
else:
    epi_dbg = pd.DataFrame([
        {
            "id": e["episode_id"],
            "count": e["features"]["count"],
            "error_ratio": e["features"]["error_ratio"],
            "restarts": e["features"]["restarts"],
            "http5xx": e["features"]["http5xx"],
            "rollout_in_window": e["features"]["rollout_in_window"],
            "ent_namespace": ",".join(e["entities"].get("namespace", [])),
            "ent_pod": ",".join(e["entities"].get("pod", [])),
            "ent_node": ",".join(e["entities"].get("node", [])),
            "start": e["start"],
            "end": e["end"],
        }
        for e in eps
    ]).sort_values(["start","id"]).reset_index(drop=True)

print("Episodes:", len(eps))
display(epi_dbg.head(12))


NameError: name 'unified' is not defined